# Tutorial 3

## Monte Carlo Tree Search

In this mini project we will implement a recently developed planning method called monte carlo tree search. This algorithm combines the strengths of a structured tree search with the exploratory behavior of random "rollouts" to focus on promising parts of the state space, and has been particularly successful with perfect information games using a large state space (such as go). Similarly to RL, planning has to balance exploration and exploitation and MCTS employs a number of heuristics to try to do it.



As you can see from the pseudocode, the algorithm has three key components: a tree policy (for action selection within the search tree), a default policy (for rollouts outside the search tree), and a backup function for backing up accumulated rewards over the simulated episode. We will first implement an example for each of these, and then put them together into the complete algorithm.

A version in the algorithm is summerized in the following pseudocode:

<img src="mcts_pseudocode.png",width='500'>




You can import a generic graph structure with state nodes and action nodes, and implement a set of classes defining corresponding states and actions using the code below, aimed to integrate RL worlds from module 2 (in this case the windy cliff world). State nodes have these state objects as attributes, which in turn have position attribute returning the location in the maze. 

Action objects have the attribute move, which tell you the direction (0 to 3) corresponding to that action. You can change, and extend the reward function later to better solve the task. Please study the code below and graph.py for more details.



In [ ]:
import numpy as np
import RL_worlds
from RL_worlds import windy_cliff_grid_2
from RL_worlds import windy_cliff_grid
from graph import *
from default_policies import RandomKStepRollOut
import random
import utils
#from backups import monte_carlo
class MazeAction(object):
    def __init__(self, move):
        self.move = np.asarray(move)
    
    def __eq__(self, other):
        return self.move == other.move
        
   
class MazeState(object):
    def __init__(self, pos):
        self.pos = np.asarray(pos)
        self.actions = [MazeAction(0),
                        MazeAction(1),
                        MazeAction(2),
                        MazeAction(3)]
    
    def perform(self, action):
        pos,r = wd.get_outcome(self.pos,action.move)
        
        return MazeState(pos)
        
    def reward(self, parent, action):
         if self.pos in [53,131]:
             
             return 10000
         elif self.pos in [2,3,4,28,42,56,70]:
             
            return -10000
         elif self.pos in rstates[len(rstates)-5:]:#== parent.pos:
             
             return -200
         return -10
         
        
        
        
    def is_terminal(self):
        if (self.pos in [2,3,4, 53,131,28,42,56,70]):
           
            return True
        else:
            return False
             
   

Once you created a search tree, you can navigate around it using the following syntax (which we highlight with a list of examples):
To see the maze location corresponding to a node: node.state.pos
To see the visitation counts: node.n
The possible actions from a state node are the children nodes node.children.values()
So to see how many times during the rollouts you took action 0 from the root node: root.children.values()[0].n

# Exercise 1


Write a function called ucbt to execute the UCB (upper confidence bound) tree poilcy. In particular write a function that takes as input an action-node, and returns the ucb value, $Q(s,a)_{ucb}=Q(s,a)+c * \sqrt {\frac{\log n(s)}{n(s,a)}}$, where n() denotes the number of visitations to the state, or action node. UCB uses optimism in the face of uncertainty, and chooses the action that could potentially be the best, given the remaining uncertainty about its value (specified in terms of the number of previous visitations). In this sense it's a bandit algorithm with Bayesian inspiration.

Write a default policy function that performs a k-step random rollout (picking a random action at each step for k steps), but stops if a state is terminal. The function should return the total accumulated discounted reward.

Write a backup function, that takes as input the terminal node and adjusts the value of all nodes (state and action) visited in the episode (this just means walking up the search tree). You can assume that the result of the rollout from the terminal node has been stored in node.reward. The rewards for actions taken while in the search tree can either be computed inside the backup function, or stored during the tree traversal during the episode.




In [ ]:

stop_k=500
c=5000

def ucbt(action_node):
     return (action_node.q +
                c * np.sqrt(2 * np.log(action_node.parent.n) /
                                 action_node.n))
 


'''
class RandomKStepRollOut(object):
   
    def __init__(self, k):
        self.k = k

    def __call__(self, state_node):
        self.current_k = 0

        def stop_k_step(state):
            self.current_k += 1
            return self.current_k > self.k or state.is_terminal()

        return _roll_out(state_node, stop_k_step)
    
def random_terminal_roll_out(state_node):
    """
    Estimate the reward with the sum of a rollout till a terminal state.
    Typical for terminal-only-reward situations such as games with no
    evaluation of the board as reward.

    :param state_node:
    :return:
    """
    def stop_terminal(state):
        return state.is_terminal()

    return _roll_out(state_node, stop_terminal)


def _roll_out(state_node, stopping_criterion):
    gamma=0.99
    state = state_node.state
    parent = state_node.parent.parent.state
    action = state_node.parent.action
    reward = state.reward(parent,action)
    #print('r',reward)
    while not stopping_criterion(state):
        #print('rr',reward)
        

        action = np.random.choice(state_node.state.actions)
        parent = state
        state = parent.perform(action)
        reward += gamma*state.reward(parent, action)
        gamma*=0.99
    return reward






def Random_k_Rollout(state_node):
    k=0
    gamma=0.99
    state = state_node.state
    parent = state_node.parent.parent.state
    action = state_node.parent.action
    reward = state.reward(parent,action)
    #print('r',reward)
    while not (k>stop_k or state.is_terminal()):
        k+=1
        
        

        action = np.random.choice(state_node.state.actions)
        parent = state
        state = parent.perform(action)
        #print state.pos
        #print state.reward(parent,action)
        reward += gamma*state.reward(parent, action)
        
        gamma*=0.99
    return reward

'''
def monte_carlo(node, depth):
    
    r = node.reward
    while node is not None:
        gamma=0.99
        disc=1
        node.n += 1
        node.q = ((node.n - 1)/node.n) * node.q + 1/node.n * r*disc
        node = node.parent
        disc*=gamma     


We will now write the core algorithm for MCTS.
<img src="basic_algorithm.png",width='600'>

We can write three helper functions:

best_child should return, given a state node and the tree policy, a successor state when taking the maximum value action.

expand should return ,given a state node as input, a successor state after randomly taking one of the untried actions.

Finally get_next_node should take as input a state node and the tree policy, and return a successor state node using best_child while inside the search tree (no untried actions), or using expand when reaching a leaf of the search tree (The state node has untried actions.)



In [ ]:


def get_next_node(state_node, tree_policy):
    #Find a node that has untried children nodes
    while not state_node.state.is_terminal():
        depth=0
        if state_node.untried_actions:
            #expand the tree 
            return expand(state_node)
        else:
            depth+=1
            state_node = best_child(state_node, tree_policy)
            
    return state_node

def best_child(state_node, tree_policy):
    #print "children" ,state_node.children[0].state.pos
    #print state_node.children.values()
    best_action_node = utils.rand_max(state_node.children.values(),
                                      key=tree_policy)
    return best_action_node.sample_state()

def expand(state_node):
    action = np.random.choice(state_node.untried_actions)
    return state_node.children[action].sample_state()



    
class MCTS(object):
    
    def __init__(self, tree_policy, default_policy, backup):
        self.tree_policy = tree_policy
        self.default_policy = default_policy
        self.backup = backup

    def __call__(self, root, n=500):
       
        depth=0
        if root.parent is not None:
            raise ValueError("Root's parent must be None.")

        for _ in range(n):
            node = get_next_node(root, self.tree_policy)
            node.reward = self.default_policy(node)
            self.backup(node,depth)

        return utils.rand_max(root.children.values(), key=lambda x: x.q).action



These helper functions can then be integrated into a callable Tree search object (MCTS here), where you assign you should input your own tree policy, default policy and backup functions at the time of initialization. utils.rand_max is provided to randomly choose between possible actions of equal value.


In [ ]:
class MCTS(object):
    
    def __init__(self, tree_policy, default_policy, backup):
        self.tree_policy = tree_policy
        self.default_policy = default_policy
        self.backup = backup

    def __call__(self, root, n=500):
       
        depth=0
        if root.parent is not None:
            raise ValueError("Root's parent must be None.")

        for _ in range(n):
            node = get_next_node(root, self.tree_policy)
            node.reward = self.default_policy(node)
            self.backup(node,depth)

        return utils.rand_max(root.children.values(), key=lambda x: x.q).action


Run your tree search on the windy cliff world problem. There are many moving parts you can adjust, including the reward structure (though you are not supposed to introduce new location specific rewards), any discounting you introduce, the constant for the UCB algorithm. You can also vary how the backup is performed.

Try to tune your algorithm so it can reach one of the gold states from any starting state.

In [ ]:
policy_c=5000
rollout_n=500

mctsa=MCTS(tree_policy=ucbt, 
            default_policy=RandomKStepRollOut(rollout_n),
            backup=monte_carlo)

In [ ]:
nsteps=1
total_steps=50
rollout_n=500
start_state=162
rstates=[start_state]
wd=windy_cliff_grid_2()
new_state=start_state
rstates=[start_state]
ractions=[]
while not MazeState(new_state).is_terminal() and nsteps<=total_steps:
    nsteps+=1
    root=StateNode(None,MazeState(new_state))
    best_action = mctsa(root)
    new_state,rreward=wd.get_outcome(root.state.pos,best_action.move)
    rstates.append(new_state)
    ractions.append(best_action)
    print('S:', new_state,'A:',best_action.move, 'R:', rreward,)